In [1]:
import cv2

In [2]:
image = cv2.imread('test_image.jpg')
cv2.imshow('input_image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Converting image to grayscale

In [3]:
import cv2
import numpy as np

In [4]:
image = cv2.imread('test_image.jpg')
lanelines_image = np.copy(image)
gray_conversion = cv2.cvtColor(lanelines_image, cv2.COLOR_BGR2GRAY)

cv2.imshow('gray_conversion', gray_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

smoothing the image

In [5]:
blur_conversion = cv2.GaussianBlur(gray_conversion, (5,5), 0)

cv2.imshow('blur_image', blur_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

Canny edge detection

In [6]:
canny_conversion = cv2.Canny(blur_conversion, 50, 155)

cv2.imshow('canny', canny_conversion)
cv2.waitKey(0)
cv2.destroyAllWindows()

Masking the region of interest

In [7]:
def canny_edge(image):
    gray_conversion = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur_conversion = cv2.GaussianBlur(gray_conversion, (5,5), 0)
    canny_conversion = cv2. Canny(blur_conversion, 50, 150)
    return canny_conversion

In [8]:
def reg_of_interest(image):
    image_height = image.shape[0]
    polygons = np.array([[(200, image_height), (1100, image_height), (550,250)]])
    image_mask = np.zeros_like(image)
    cv2.fillPoly(image_mask, polygons, 255)
    return image_mask

In [9]:
image = cv2.imread('test_image.jpg')
lanelines_image = np.copy(image)
canny_conversion = canny_edge(lanelines_image)

In [10]:
cv2.imshow('result', reg_of_interest(canny_conversion))
cv2.waitKey(0)
cv2.destroyAllWindows()

applying bitwise_and

In [11]:
import matplotlib.pyplot as plt

In [12]:
def reg_of_interest(image):
    image_height = image.shape[0]
    polygons = np.array([[(200, image_height), (1100,image_height), (550,250)]])
    image_mask = np.zeros_like(image)
    cv2.fillPoly(image_mask, polygons, 255)
    masking_image = cv2.bitwise_and(image, image_mask)
    return masking_image

In [13]:
image = cv2.imread('test_image.jpg')
lanelines_image = np.copy(image)
canny_conversion = canny_edge(lanelines_image)
cropped_image = reg_of_interest(canny_conversion)

In [14]:
cv2.imshow('result', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

hough transform

In [15]:
def show_lines(image, lines):
    lines_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(lines_image, (x1, y1), (x2,y2), (255,0,0), 10)
    return lines_image

In [16]:
image = cv2.imread('test_image.jpg')
lanelines_image = np.copy(image)
canny_conv = canny_edge(lanelines_image)
cropped_image = reg_of_interest(canny_conv)
lane_lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
final_image = show_lines(lanelines_image, lane_lines)

In [17]:
cv2.imshow('final_image', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Detect Road marking in images#

In [18]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * (3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

In [19]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameter = np.polyfit((x1,x2), (y1,y2), 1)
        slope = parameter[0]
        intercept = parameter[1]
        if slope<0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    #print('left_fit', left_fit)
    left_fit_average = np.average(left_fit, axis=0)
    #print('left_fit_averagge', left_fit_average)
    right_fit_average = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, left_fit_average)
    #print('left_line',left_line)
    right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])

        

In [20]:
image = cv2.imread('test_image.jpg')
lanelines_image = np.copy(image)
canny_image = canny_edge(lanelines_image)
cropped_image = reg_of_interest(canny_image)
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
averaged_lines = average_slope_intercept(lanelines_image, lines)
line_image = show_lines(lanelines_image, averaged_lines)
combined_image = cv2.addWeighted(lanelines_image, 0.8, line_image, 1, 1)

In [21]:
cv2.imshow('result', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
print(lines)

[[[658 371 755 468]]

 [[696 417 747 470]]

 [[299 701 386 567]]

 [[747 472 969 702]]

 [[331 684 452 481]]

 [[588 299 660 371]]

 [[811 526 868 582]]

 [[615 332 704 424]]

 [[900 613 976 689]]

 [[401 542 454 461]]

 [[351 675 379 625]]

 [[704 418 839 553]]

 [[592 304 704 416]]

 [[358 609 409 532]]]


In [23]:
cv2.imshow('result', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
print(averaged_lines)

[[309 704 484 422]
 [985 704 705 422]]


In [25]:
cv2.imshow('result', line_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
cv2.imshow('result', canny_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
print(image.shape)

(704, 1279, 3)


In [28]:
cv2.imshow('result', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# detecting road markings in video

In [29]:
cap = cv2.VideoCapture('test2.mp4')

In [30]:
while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = canny_edge(frame)
    cropped_canny = reg_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    average_lines = average_slope_intercept(frame, lines)
    line_image = show_lines(frame, average_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow('result', combo_image)
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindow()

C:\Users\yash\Anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\yash\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: cannot unpack non-iterable numpy.float64 object